In [45]:
import requests
import pandas as pd


In [46]:
API_Token = "780ed9d1-727c-410f-b73e-2bed8b05c11a"
API_Secret = "9bdf99f4de7a672d0768761df407ab60"

In [47]:
BASE_URL = "https://carapi.app/api"

def get_jwt_token():
    login_url = f"{BASE_URL}/auth/login"
    credentials = {
        "api_token" : API_Token,
        "api_secret" : API_Secret
    }
    response = requests.post(login_url,json=credentials)
    response.raise_for_status()
    return response.text

jwt_token = get_jwt_token()

In [48]:
def get_trim_ids_for_make_model(make, model, year):
    url = f"{BASE_URL}/trims"
    headers = {'Authorization': f'Bearer {jwt_token}'}
    params = {'make': make, 'model': model, 'year': year}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()

    data = response.json()

    trim_ids = [trim['id'] for trim in data['data']]  # Adjusted to access the 'trims' key

    return trim_ids


def get_trim_details(trim_id):
    url = f"{BASE_URL}/trims/{trim_id}"
    headers = {'Authorization': f'Bearer {jwt_token}'}
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    return response.json()

testcar = get_trim_ids_for_make_model("Subaru", "Forester", 2016)

for car in testcar:
    
    print(get_trim_details(car)['msrp'])

29195
33795
28795
25295
25795
22395
23395
30795


In [99]:
df = pd.read_csv("merged_dataset.csv")

df["model_make_id"] = pd.NA #
df["model_name"] = pd.NA #
df["model_trim"] = pd.NA #
df["model_year"] = pd.NA #
df["model_body"] = pd.NA #
df["model_engine_cyl"] = pd.NA #
df["model_engine_fuel"] = pd.NA #
df["model_drive"] = pd.NA #
df["model_transmission_type"] = pd.NA #
df["model_seats"] = pd.NA #
df["model_weight_kg"] = pd.NA #
df["model_fuel_cap_l"] = pd.NA #

for index, row in df.iterrows():
    if index < 4487:
        df.at[index, 'reg_date'] = row['reg_date'][:-2] + '20' + row['reg_date'][-2 :]

    else:
        df.at[index, 'reg_date'] = row['reg_date'][:11]



count = 0
for index, row in df.iterrows():
    car_name = row['model']
    if type(row['reg_date']) != float:
        car_year = row['reg_date'][-4:]
    words = car_name.split()
    if len(words) > 1:
        car_brand = words[0]
        car_model = words[1]
        car_ids = get_trim_ids_for_make_model(car_brand , car_model, car_year)
        if car_ids:
            car_details = get_trim_details(car_ids[0])
            if car_details['make_model_id'] and car_details['make_model']['name'] and car_details['id'] and car_details['year'] and car_details['make_model_trim_body']['type'] and car_details['make_model_trim_body']['seats'] and car_details['make_model_trim_body']['curb_weight'] and car_details['make_model_trim_engine']['fuel_type'] and car_details['make_model_trim_engine']['cylinders'] and car_details['make_model_trim_engine']['drive_type'] and car_details['make_model_trim_engine']['transmission'] and car_details['make_model_trim_mileage']['fuel_tank_capacity']:
                count += 1
                df.at[index, 'model_make_id'] = car_details['make_model_id']
                df.at[index, 'model_name'] = car_details['make_model']['name']
                df.at[index, 'model_trim'] = car_details['id']
                df.at[index, 'model_year'] = car_details['year']
                df.at[index, 'model_body'] = car_details['make_model_trim_body']['type']
                df.at[index, 'model_seats'] = car_details['make_model_trim_body']['seats']
                df.at[index, 'model_weight_kg'] = car_details['make_model_trim_body']['curb_weight'] / 2.205
                df.at[index, 'model_engine_fuel'] = car_details['make_model_trim_engine']['fuel_type']
                df.at[index, 'model_engine_cyl'] = car_details['make_model_trim_engine']['cylinders']
                df.at[index, 'model_drive'] = car_details['make_model_trim_engine']['drive_type']
                df.at[index, 'model_transmission_type'] = car_details['make_model_trim_engine']['transmission']
                df.at[index, 'model_fuel_cap_l'] = float(car_details['make_model_trim_mileage']['fuel_tank_capacity']) * 3.785

  
print(count, " matches found")
    
    
    
    


3294  matches found


In [100]:
df.to_csv('test_results.csv', index=False)

In [105]:
count = 0
for index, row in df.iterrows():
    if pd.notna(row['model_body']):
        if 'hidden' not in row['model_body']:
            count += 1
print(count)

2137
